# TP2 : Apprentissage multi-label sur des données textuelles

*Jordan Dutel et Ariane Paradan*

---
---

## I. Import des librairies

In [9]:
import importlib
import function_TAA as taa
importlib.reload(taa)

<module 'function_TAA' from '/home/jordan/Documents/Etudes/Master_Bio_info/Master_2/Semestre_1/TAA/TP/TP2_MLL_NLP/function_TAA.py'>

## II. Load data

In [3]:
df = taa.load_data('PubMed-multi-label-dataset.csv', sep=',')

                                                   Title  \
0      Expression of p53 and coexistence of HPV in pr...   
1      Vitamin D status in pregnant Indian women acro...   
2      [Identification of a functionally important di...   
3      Multilayer capsules: a promising microencapsul...   
4      Nanohydrogel with N,N'-bis(acryloyl)cystine cr...   
...                                                  ...   
49995  Five donors-one recipient: modeling a mosaic o...   
49996  The role of eicosanoids in cyclosporine nephro...   
49997  Impact of pancreaticoduodenal arcade dilation ...   
49998  Outcomes of Preterm Infants following Discussi...   
49999  Molecular subtyping of Borrelia burgdorferi se...   

                                            abstractText  \
0      Fifty-four paraffin embedded tissue sections f...   
1      The present cross-sectional study was conducte...   
2      The occurrence of individual amino acids and d...   
3      In 1980, Lim and Sun introduced 

## III. Pre-processing

### A. Data infos

In [ ]:
taa.get_info(df)

### B. Data cleaning

La fonction est longue à faire tourner (~4 min)

In [4]:
df = taa.clean_text(df, 'abstractText')

### C. Features and labels split

In [5]:
target_column = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'Z']
X, y = taa.split_feature_label(df, target_column, multilabel=True)

Les features sont dans le dataframe X (50000 lignes, 6 colonnes).
La target est dans le dataframe y (50000 lignes, 14 colonnes).


### D. Split train test

In [6]:
X_train, X_test, y_train, y_test = taa.split_train_test(X, y, test_size=0.5, random_state=42)

Le dataset a été divisé en training et testing sets avec 50.0% des données dans le testing set.
X_train : 25000 lignes, 6 colonnes (features)
X_test : 25000 lignes, 6 colonnes (features)
y_train : 25000 lignes, 14 colonnes (labels)
y_test : 25000 lignes, 14 colonnes (labels)


In [7]:
X_train_vec, X_test_vec, feature_names = taa.tfidf_vectorize(X_train['abstractText'], X_test['abstractText'], max_features=2000)

Les textes ont été vectorisés en 2000 features.
X_train : 25000 lignes, 2000 colonnes (features)
X_test : 25000 lignes, 2000 colonnes (features)

Feature_names - Vocabulaire : 

['aa' 'abdominal' 'ability' 'able' 'abnormal' 'abnormalities' 'absence'
 'absolute' 'absorption' 'access' 'accompanied' 'according' 'account'
 'accumulation' 'accuracy' 'accurate' 'acetate' 'achieve' 'achieved'
 'acid' 'acids' 'acquired' 'acquisition' 'act' 'action' 'actions'
 'activated' 'activation' 'active' 'activities' 'activity' 'acute' 'ad'
 'adaptation' 'added' 'addition' 'additional' 'additionally' 'address'
 'adenosine' 'adequate' 'adherence' 'adhesion' 'adjacent' 'adjusted'
 'adjustment' 'administered' 'administration' 'admission' 'admitted'
 'adolescents' 'adult' 'adults' 'advanced' 'adverse' 'affect' 'affected'
 'affecting' 'affects' 'affinity' 'age' 'aged' 'agent' 'agents' 'ages'
 'aging' 'agonist' 'agreement' 'aim' 'aimed' 'aims' 'air' 'airway' 'al'
 'albumin' 'alcohol' 'algorithm' 'allele' 'allow

30min pour run le ClassifierChain

In [10]:
results = taa.run_models(X_train_vec, X_test_vec, y_train, y_test)

Running MultiOutputClassifier...
Evaluating MultiOutputClassifier...
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.73      0.76     11641
           1       0.95      0.99      0.97     23274
           2       0.86      0.83      0.84     13179
           3       0.90      0.89      0.89     15486
           4       0.81      0.96      0.88     19629
           5       0.84      0.58      0.69      4437
           6       0.82      0.88      0.85     16802
           7       0.57      0.09      0.15      3048
           8       0.72      0.40      0.51      2839
           9       0.72      0.23      0.34      2723
          10       0.73      0.32      0.45      3729
          11       0.86      0.86      0.86     10623
          12       0.81      0.75      0.78     11447
          13       0.73      0.50      0.60      4024

   micro avg       0.85      0.81      0.83    142881
   macro avg       0.79      0.64      0.6

In [10]:
svd, svd_matrix_train, svd_matrix_test = taa.extract_concept_SVD(2, X_train_vec, X_test_vec)

In [11]:
# Afficher les 5 mots les plus pertinents pour chaque concept
taa.print_top_words(svd, feature_names, 5)

Concept #0: patients cells group study treatment
Concept #1: patients group years clinical risk



In [12]:
results = taa.run_models(svd_matrix_train, svd_matrix_test, y_train, y_test)

Running MultiOutputClassifier...
Evaluating MultiOutputClassifier...
Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.52      0.59     11641
           1       0.93      1.00      0.96     23274
           2       0.76      0.68      0.72     13179
           3       0.79      0.82      0.80     15486
           4       0.79      1.00      0.88     19629
           5       0.00      0.00      0.00      4437
           6       0.74      0.90      0.81     16802
           7       0.00      0.00      0.00      3048
           8       0.00      0.00      0.00      2839
           9       0.00      0.00      0.00      2723
          10       0.00      0.00      0.00      3729
          11       0.81      0.75      0.78     10623
          12       0.73      0.67      0.70     11447
          13       0.23      0.00      0.00      4024

   micro avg       0.79      0.71      0.75    142881
   macro avg       0.46      0.45      0.4

Définition des métriques
- Micro-F1 : 
    - Moyenne pondérée des F1-scores, calculée sur l'ensemble des échantillons et des labels. Elle accorde plus de poids aux classes fréquentes.
    - Indique la performance globale, favorisant les labels fréquents.

- Macro-F1 : 
    - Moyenne des F1-scores par classe, non pondérée par la fréquence des classes.
    - Évalue l'équilibre des performances sur toutes les classes, même les rares.

- Zero-One-Loss : 
    - Proportion d'échantillons mal prédits entièrement (aucun label correct pour ces échantillons).
    - Plus cette valeur est basse, plus la méthode est précise dans ses prédictions globales.


---


MultiOutputClassifier Metrics :
- Sans extraction de concept :
    - micro-F1: 0.8282540933682061
    - macro-F1: 0.6836620901639565
    - zero-one-loss: 0.86224
- Avec extraction de concept :
    - micro-F1: 0.7492527939410732
    - macro-F1: 0.44628391519437743
    - zero-one-loss: 0.93528

ClassifierChain Metrics:
- Sans extraction de concept :
    - micro-F1: 0.7800861548030066
    - macro-F1: 0.6245708592773794
    - zero-one-loss: 0.91156
- Avec extraction de concept :
    - micro-F1: 0.7323635473724367
    - macro-F1: 0.48255564574963455
    - zero-one-loss: 0.94308


---


Micro-F1 et Macro-F1 diminuent :
- La réduction de dimensions via TruncatedSVD réduit le détail et la richesse de l’information disponible pour la classification. Cela peut entraîner une perte d’efficacité, notamment pour des labels rares ou spécifiques.
- Micro-F1 :
    - Une diminution de la Micro-F1 signifie que les modèles deviennent moins performants sur les classes fréquentes. Cela peut arriver parce que :
        - La réduction de dimensions élimine des informations discriminantes importantes pour séparer correctement les classes.
        - Les données simplifiées via SVD ne contiennent plus suffisamment de diversité pour capturer les relations spécifiques aux labels fréquents.
- Macro-F1 :
    - La baisse de la Macro-F1 signifie que les performances se détériorent également sur les classes rares. Cela peut indiquer que :
        - Les classes peu fréquentes perdent encore plus d’information lors de la réduction de dimensions.
        - Les modèles ont du mal à distinguer des concepts fins ou spécifiques à cause de l’agrégation des données (SVD regroupe plusieurs mots synonymes ou termes proches dans des "concepts")

Zero-One-Loss augmente :
- Une augmentation de la zero-one-loss signifie que davantage d’échantillons n’ont aucun label correctement prédit. Cela peut s'expliquer par :
    - La simplification excessive des données via SVD, qui rend plus difficile pour les modèles de capturer des associations précises entre les textes et leurs labels.
    - Les concepts générés par SVD peuvent introduire de la confusion, en regroupant des termes ou des relations qui étaient distinctes dans les données originales.

Différence entre MOC et ECC :
- MultiOutputClassifier a de meilleures performances générales (Micro-F1, Macro-F1) car il considère chaque label de manière indépendante. En revanche, ClassifierChain peut souffrir si les premières étapes de la chaîne introduisent des erreurs.


---


Impact de l'extraction de concepts
- Réduction de la richesse des données :
        - La représentation via SVD simplifie les données en les projetant dans un espace de dimensions réduites. Bien que cela puisse résoudre des problèmes comme la synonymie ou la polysémie, cela peut aussi éliminer des détails importants pour la classification, surtout dans des tâches complexes comme le multi-label.
- Perte de granularité :
        - Les "concepts" créés par SVD sont des abstractions qui peuvent être utiles pour des tâches générales (comme la recherche d’information ou le regroupement thématique). Cependant, pour une tâche supervisée, cette abstraction peut rendre les données moins spécifiques et donc moins informatives pour les modèles.